## Create Points for StreamStats

__Description__:

__Input__:

__Output__:


*Authors*: sputnam@Dewberry.com & slawler@Dewberry.com

#### Load libraries and Python options:

In [1]:
import os
import collections 
import pandas as pd
import numpy as np
import geopandas as gpd
from osgeo import gdal, ogr,osr
from shapely.geometry import Point
from StreamStats_Points import*

#### Load the masked stream grid:

In [2]:
in_tif=r'C:\Users\sputnam\Documents\GitHub\usgs-tools\results\rock_creek_clip.tif' #Load the stream grid raster which was masked by the catchment polygon

In [3]:
sg = StreamGrid(in_tif) #Open the stream grid raster and create an object

crs=sg.crs_value() #Extract the coordinate reference system value (epsg) for the raster
print("epsg:",crs) #Print the value

epsg: 5070


In [4]:
df = sg.dataframe() #Create a dataframe from the stream grid data
df.replace(255, 0, inplace=True) #Replace 255 with 0, where 255 corresponds to the non-stream cells
df.head(n=2) 

,0,1,2,3,4,5,6,7,8,9,...,1205,1206,1207,1208,1209,1210,1211,1212,1213,1214
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Specify the pour point to set the start location of the search:

In [5]:
lat=1925315.186 #latitude of the pourpoint at the catchment outlet
lon=1616784.964 #longitude of the pourpoint

In [6]:
pix_x, pix_y =coord2index(sg, lat, lon) #Transform the lat and lon values to the row/column location with the stream grid dataframe
pourpoint=[(pix_x, pix_y)] #Add these values to a list as a touple
print(pourpoint)

[(877, 1848)]


In [7]:
cellsize=sg.cell_size()
print("The Cell Size:",cellsize)

The Cell Size: 10.0


### Move up the stream and identify the confluences:

In [8]:
def MoveUpstream(df, stream_cell, nogo):
    """This function searches the 8 cells surrounding it to determine the location of the next stream cell(s).
        Arguments: df=the dataframe containing the stream raster data, stream_cell=the current stream cell that will be used to search for the next stream cell,
        nogo=list of stream cells that do not want to return as a new stream cell
    """
    row=stream_cell[0] #Extract the raster row 
    col=stream_cell[1] #Extract the raster column
    
    cell_value= df[row][col] #Determine the value of the cell
    assert cell_value == 1, "The provided cell in MoveUpstream is not a stream cell" #The value of the cell must be equal to the value associated with a stream cell
    
    nogo.append((row,col))   

    stream_cell=[]
    
    for i in range(-1,2): #For -1, 0, 1 in the vertical direction (move up and down rows)
        for j in range(-1,2): #For -1, 0, 1 in the horizontal direction (move across columns)
            value = df[row + i][col+j] #Read value of raster cell
            if value == 0: # if value is zero, no stream in this cell
                continue #loop back
            elif value==1 and (row+i,col+j) not in nogo: # if value is 1 and the cell is not in the nogo list, then...
                stream_cell.append((row + i,col+j)) #Add the new cell or cells to the stream_cell list
    return stream_cell, nogo    

In [9]:
def FindConfluence(df, stream_cell, nogo, mainstem, dis, DisExl): #Keep moving upstream until you find a confluence--problem: might hit a dead end, maybe add an else if it is the end?
    """Function which repeates the MoveUpstream function until it finds two or more stream cells surrounding the provided stream_cell, indicating a confluence
    """
    while len(stream_cell)==1: #While there is only 1 stream cell returned as we move up stream, keep moving up stream
        cell1=stream_cell[0]
        stream_cell, nogo=MoveUpstream(df, stream_cell[0], nogo)
        
        if len(stream_cell)>=1:
            cell2=stream_cell[0]
        else:
            cell2=cell1
            
        cell_dis=TrueDistance(cell1, cell2, cellsize)
        dis=dis+cell_dis
        if len(stream_cell)!=0:
            dis_lower=dis/DisExl-10.0/DisExl
            dis_upper=dis/DisExl+10.0/DisExl  
            if dis_lower<=int(dis/DisExl)<=dis_upper:
                mainstem.append(stream_cell[0])
    if len(stream_cell)>1: #If the number of stream cells 
        nogo=list(set(nogo+stream_cell))
        
    return stream_cell, nogo, mainstem, dis

In [10]:
def CheckTribDis(stream_cell, nogo, DisExl):
    """
    """
    nogo_temp=nogo.copy()
    mainstem_temp=[]
    remove=[]

    for cell in stream_cell:
        dis_temp=0.0
        stream_cell1, nogo_temp, mainstem_temp, dis_temp=FindConfluence(df, [cell], nogo_temp, mainstem_temp, dis_temp, DisExl)
        if len(stream_cell1)==0 and dis_temp<DisExl:
            remove.append(cell)
    stream_cell=list(set(stream_cell) - set(remove))
    return stream_cell

In [11]:
def CheckConfluence(df, stream_cell, nogo, mainstem, dis, DisExl):
    """
    """
    while len(stream_cell)==1:
        stream_cell, nogo, mainstem, dis=FindConfluence(df, stream_cell, nogo, mainstem, dis, DisExl)
        if len(stream_cell)>1:
            stream_cell=CheckTribDis(stream_cell, nogo, DisExl)
    return stream_cell, nogo, mainstem, dis

In [88]:
nogo=[] #Empty list to store the nogo points, i.e. the stream cells we previously searched
mainstem=[]
Incl_Confl=[]
dis=0.0

#cnum=0 #The confluence ID, start it at zero
#snum=0 #The confluence number at a specific ID, start it at zero
#cellnum=(cnum,snum) #Combine the confluence ID and number into a signle touple

#confl=collections.OrderedDict() #Create an empty ordered dictionary to store the confluence information
#confl[cnum]={"pts":pourpoint,"npts":len(pourpoint)} #Add the pourpoint as a confluence to this ditionary, specifying the pourpoint as "pts" and the number of points as "npts"

#confl3=[] #Empty list to store the confluence locations that are three cells from the original junction. 

DisExl=(5280/2.0)/3.28084

In [89]:
stream_cell=pourpoint

In [90]:
confl, nogo, mainstem, dis=CheckConfluence(df, stream_cell, nogo, mainstem, dis, DisExl)

In [86]:
dis_confl=0.0
next_confl={}

for i in range(len(confl)):
    stream_cell, nogo, mainstem, dis_confl=CheckConfluence(df, [confl[i]], nogo, mainstem, dis_confl, DisExl) 
    if len(stream_cell)==0 and dis_confl

In [86]:
    next_confl[i]={'confl': stream_cell, 'dis':dis_confl}

In [74]:
next_confl

{0: {'confl': [], 'dis': 872.5483399593904},
 1: {'confl': [(848, 1632), (849, 1633)], 'dis': 464.5584412271571}}

In [76]:
for k in next_confl.keys():
    if next_confl[k]['confl']==[] and next_confl[k]['dis']>=DisExl
        Incl_Confl
    

0
1


In [87]:
confl

[(829, 1671), (830, 1669)]

In [ ]:
for k, v in a.items():
    print(k,v)

In [ ]:
def NextConfluence(df, confl, cellnum, nogo):
    """
    """
    stream_cell, nogo=MoveUpstream(df, confl[cellnum[0]]['pts'][cellnum[1]], nogo) #Move up one cell from the confluence
    if len(stream_cell)==1:
        stream_cell, nogo=MoveUpstream(df, stream_cell[0], nogo) #And move up one more time 
        if len(stream_cell)==1:
            if 'confl' not in confl[cellnum[0]]: #Add the stream_cell that is located three up from the intial split point.
                confl[cellnum[0]]['confl']=collections.OrderedDict()
                confl[cellnum[0]]['confl'][stream_cell[0]]={}
            else:
                confl[cellnum[0]]['confl'][stream_cell[0]]={}
            nextconfl, nogo, dis=FindConfluence(df, stream_cell, nogo) #Conf1 is empty if end 
            assert len(nextconfl)!=1, "Expecting either a confluence or the end of a tributary"
            if len(nextconfl)>1:
                confl[max(list(confl.keys()))+1]={'pts':nextconfl, 'npts':len(nextconfl)}
                confl[cellnum[0]]['confl'][stream_cell[0]]['Trib']=0
                confl[cellnum[0]]['confl'][stream_cell[0]]['Dis']=dis
            elif len(nextconfl)==0:
                confl[cellnum[0]]['confl'][stream_cell[0]]['Trib']=1
                confl[cellnum[0]]['confl'][stream_cell[0]]['Dis']=dis
        elif len(stream_cell)>1:
            confl[cellnum[0]]['pts']=confl[cellnum[0]]['pts']+stream_cell
            confl[cellnum[0]]['npts']=len(confl[cellnum[0]]['pts'])
    elif len(stream_cell)>1:
        confl[cellnum[0]]['pts']=confl[cellnum[0]]['pts']+stream_cell
        confl[cellnum[0]]['npts']=len(confl[cellnum[0]]['pts'])
    return confl, nogo    

In [ ]:
l=len(confl.keys())-1 
j=0
while j<=l: 
    k=confl[j]['npts']-1 
    i=0 
    while i<=k: 
        confl, nogo=NextConfluence(df, confl, (j,i), nogo)
        k=confl[j]['npts']-1 
        i+=1 
    l=len(confl.keys())-1 
    j+=1 

In [ ]:
ppoints=[] #Empty list to store the row and column of each confluence
for i in range(len(confl.keys())):
    ppoints=ppoints+confl[i]['pts']
    
confl3=[] #Empty list to store the row and column of each confluence point that is three cells away from the original junction

for i in range(len(confl.keys())):
    if 'confl' in confl[i]:    
        if len(confl[i]['confl'])>1:
            confl3=confl3+list(confl[i]['confl'].keys())

In [ ]:
Trib=[] #Identified as a tributary if the stream cells has no confluence points

for i in range(len(confl.keys())):
    if 'confl' in confl[i]:    
        if len(confl[i]['confl'])>0:
            for j in list(confl[i]['confl'].keys()):
                if confl[i]['confl'][j]['Trib']==1:  
                    Trib=Trib+[(j)]

In [ ]:
TribExl=[] # If the point is for a tributary and the distance is less than 1/2 mile, add it to an exlusion list


for i in range(len(confl.keys())):
    if 'confl' in confl[i]:    
        if len(confl[i]['confl'])>0:
            for j in list(confl[i]['confl'].keys()):
                if confl[i]['confl'][j]['Trib']==1 and confl[i]['confl'][j]['Dis']<DisExl:
                    TribExl=TribExl+[(j)]

confl3select=[] #The list of non-excluded confluence points

for i in range(len(confl3)):
    if confl3[i] not in TribExl:
        confl3select.append(confl3[i])                    

In [ ]:
len(confl3select)

### Save the conflunces as a shapefile:

In [ ]:
files=[ppoints,confl3, Trib, confl3select]
names=['ppoints','confl3', 'Trib', 'confl3select']
for i in range(len(files)):
    longitude, latitude=index2coord(sg, files[i])
    gdf=geodataframe(longitude, latitude, crs)
    gdf.to_file(filename = r'C:\Users\sputnam\Documents\GitHub\usgs-tools\results\%s.shp'%names[i])

#gdf.head(2)

In [ ]:
longitude, latitude=index2coord(sg, mainstem)
gdf=geodataframe(longitude, latitude, crs)
gdf.to_file(filename=r'C:\Users\sputnam\Documents\GitHub\usgs-tools\results\mainstem.shp')

In [75]:
a = {'a':1, 'b':2, 'c':3}

for k, v in a.items():
    print(k,v)

a 1
b 2
c 3
